In [ ]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [ ]:
# Dependencies
import numpy as np
import pandas as pd

In [ ]:
import tensorflow
tensorflow.keras.__version__

In [ ]:
houses = pd.read_csv('Resources/home_data.csv')

cut_labels_10 = ['<125k', '125-150k', '150k-175k', '175k-200k', '200k-220k', '220k-240k', '240k-260k', '260k-280k', '280k-300k', '300k+']
cut_bins = [0, 125000, 150000, 175000, 200000, 220000, 240000, 260000, 280000, 300000, 10000000]
houses["price_range"] = pd.cut(houses['price'], bins=cut_bins, labels=cut_labels_10)

houses.head(20)


In [ ]:
# Filter out expensive houses here if we want to 


In [ ]:

X_trimmed = houses[["bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode"]]
X_trimmed = houses.drop(["id","date","price","lat","long","sqft_living15","sqft_lot15","price_range"], axis=1)
y = houses["price"]
y_ranges = houses["price_range"]

In [ ]:
y.describe()


In [ ]:
y_ranges.describe()

In [ ]:
y.plot(kind="hist")

In [ ]:
affordable = houses[houses["price"] <= 1000000]["price"]
affordable.plot(kind="hist")

In [ ]:
affordable.describe()

In [ ]:
houses.shape

## Data Pre-Processing

In [ ]:
#X = houses.drop(["price", "date"], axis=1)
y = houses["price"].values.reshape(-1,1)
print(X_trimmed.shape, y.shape)


In [ ]:
X_trimmed.head()

In [ ]:
y = houses["price"]
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_trimmed, y_ranges, random_state=42)

In [ ]:
#X_scaler = MinMaxScaler().fit(X_train)
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [ ]:
y_train

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_ranges)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


In [ ]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical

# Create a Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=14))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    #X_train_scaled,
    #y_train_categorical,
    X_train,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

In [ ]:
model.save("Trained_Models/chris_housing_trained_v1.01.h5")

In [ ]:
model = load_model("Trained_Models/chris_housing_trained_v1.h5")

## Quantify our Trained Model

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

## Make Predictions

In [ ]:
encoded_predictions = model.predict_classes(X_test[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
X_test[:20]
encoded_predictions

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test_categorical[:20])}")

In [ ]:
to_categorical y_test_categorical[:20]

In [ ]:
label_encoder.inverse_transform(y_test_categorical[:20])